In [26]:
# %%
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os
from eth_account import Account

# %%
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
print(mnemonic)

# %%
# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
from web3 import Web3
from web3.middleware import geth_poa_middleware
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import wif_to_key
from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


w3.isConnected()


# %%
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --numderive={numderive} --coin={coin} --format=jsonpretty'

# YOUR CODE HERE
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)


# %%
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {'eth':derive_wallets(mnemonic, coin=ETH, numderive=3), 'btc-test': derive_wallets(mnemonic, coin=BTCTEST, numderive=3)}


# %%
# child account selection with dictionary 
eth_key = coins['eth'][0]['privkey']
btc_key = coins['btc-test'][0]['privkey']

# %%
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    global account
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    


# %%
eth_acc = btc_acc = priv_key_to_account(ETH, eth_key)

# %%
btc_acc = priv_key_to_account(BTCTEST, btc_key)

# %%
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    global tx_data
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value":amount}
        )
        tx_data={
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransaction(account.address),
        }
        return tx_data
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        

# %%
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, recipient, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)



garden tell timber fan auction outside breeze glory hold nuclear army delay


In [3]:
derive_wallets(mnemonic, ETH,3)

[{'path': "m/44'/60'/0'/0/0",
  'address': '0x907b31E07E8D9c5504Cc1bf28C27Ec28bF8BE111',
  'xprv': 'xprvA3U1QLWpWgFJB9DcmXfC1hQtVfaR1ssQ1K6pFSNtdMUWHgbsBuBYBcqqRTFbzNnE4m26gxQ7NXJmhngDKPMJjbQ9mjGYgJ2tNrRJoiQVaoy',
  'xpub': 'xpub6GTMor3iM3obPdJ5sZCCNqMd3hQuRLbFNY2R3pnWBh1VAUw1jSVnjRAKGgf3irEVyzvJXn5g29gyiCwACjF8nYuTAXea5NiLbK1AobkXoRW',
  'privkey': '0xeb9cdd720fb80d2ab2963f5598bd756a3ea052071ef0f0b7726a0c861112790f',
  'pubkey': '021571195a3f19ccd1263ac2b9d489154d629e3b39027689294317670690f9ef87',
  'pubkeyhash': 'aab356d20238f2b99002dd643cf6e351bd58b4ba',
  'index': 0},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0x46d7598F561d976F8a122aEcbc98c346C66Fd46c',
  'xprv': 'xprvA3U1QLWpWgFJCMbAEhDotSX8s3HfKXqtu5MeNvsyCWEoUnQjJeSTAJEckZyvEowNRp1mB6AzChyLo4dcPJRmpKtGfcqcNsMz4HR4W4yMbWm',
  'xpub': 'xpub6GTMor3iM3obQqfdLikpFaTsR589izZkGJHFBKHakqmnMajsrBkhi6Z6bp4K7V1uEyyA92nMZy42qMQKUyuqgBmy4Z5ZUDXj6GNiLMdZhTc',
  'privkey': '0xf7c4e71c221f93cc424d6f0665d0ef8805ca27218981e4f6b29b55168888998b',

In [4]:
print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "n4QQa6X49seH4oK6Y6ANk66aWsFYGjbF8Y",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cSN84TRMpK2aGr3iiUq4iSgitAcB989RyMppzvmPGjWVYHuMf1bp",
            "pubkey": "03eef32169662c6839fb2ec6813474b360984e7589a6f81419e5b6c225a1403fef",
            "pubkeyhash": "fb0dea774eda06d337113eda0dcce3aa267c39a0",
            "xprv": "tprv8kVcZDTn3PwgviZRmEnrC1L69zAmvrD4cGLeb4Fu21sJ5nvELyUC3RP262JY5GEX7wrieyVBhQREZXKfe8omZ476NP3QjQetzksGUkWyQfG",
            "xpub": "tpubDHBehdW2BmdMpBbDetTSbQzCj1gi6BPyBZwRsaJCSHfgvHAzyNHnDuztGBvZNuMSi1nzkSahzzZaCxvUHi3fukMMnYJeWswgNqLtShx2CZp"
        },
        {
            "address": "mtnxNt9cnFLxwEVFykWKpraHgsutBMjYJc",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cTkXZsyCsoeyJbymN5CCi5Ty4Z2MRLjuKXsQ4Y8o7K5ogUs2xCKu",
            "pubkey": "03d728bcb8d9c68dd8305ba77006a0528673139894bb0cea3beca8b44889f1b93b",
            "pubkeyh

In [5]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0xeb9cdd720fb80d2ab2963f5598bd756a3ea052071ef0f0b7726a0c861112790f"))
w3.eth.getBalance("0x907b31E07E8D9c5504Cc1bf28C27Ec28bF8BE111")

0

In [6]:
print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "n4QQa6X49seH4oK6Y6ANk66aWsFYGjbF8Y",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cSN84TRMpK2aGr3iiUq4iSgitAcB989RyMppzvmPGjWVYHuMf1bp",
            "pubkey": "03eef32169662c6839fb2ec6813474b360984e7589a6f81419e5b6c225a1403fef",
            "pubkeyhash": "fb0dea774eda06d337113eda0dcce3aa267c39a0",
            "xprv": "tprv8kVcZDTn3PwgviZRmEnrC1L69zAmvrD4cGLeb4Fu21sJ5nvELyUC3RP262JY5GEX7wrieyVBhQREZXKfe8omZ476NP3QjQetzksGUkWyQfG",
            "xpub": "tpubDHBehdW2BmdMpBbDetTSbQzCj1gi6BPyBZwRsaJCSHfgvHAzyNHnDuztGBvZNuMSi1nzkSahzzZaCxvUHi3fukMMnYJeWswgNqLtShx2CZp"
        },
        {
            "address": "mtnxNt9cnFLxwEVFykWKpraHgsutBMjYJc",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cTkXZsyCsoeyJbymN5CCi5Ty4Z2MRLjuKXsQ4Y8o7K5ogUs2xCKu",
            "pubkey": "03d728bcb8d9c68dd8305ba77006a0528673139894bb0cea3beca8b44889f1b93b",
            "pubkeyh

In [7]:
priv_key_to_account(BTCTEST, btc_key)

<PrivateKeyTestnet: n4QQa6X49seH4oK6Y6ANk66aWsFYGjbF8Y>

In [9]:
create_tx(BTCTEST, btc_acc, "mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt", 0.0001)

'{"unspents":[{"amount":100000,"confirmations":0,"script":"76a914fb0dea774eda06d337113eda0dcce3aa267c39a088ac","txid":"8bf705557fcd093d395f5dafdaea613a4c28831ecd6c01ff3277477845078a6a","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt",10000],["n4QQa6X49seH4oK6Y6ANk66aWsFYGjbF8Y",87740]]}'

In [13]:
create_tx(BTCTEST, btc_acc, "mtnxNt9cnFLxwEVFykWKpraHgsutBMjYJc", 0.0001)

'{"unspents":[{"amount":78644,"confirmations":0,"script":"76a914fb0dea774eda06d337113eda0dcce3aa267c39a088ac","txid":"63593a15642014e511736e2bdd683f367ba9d94bc5fefec9d087ed31d348bde4","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mtnxNt9cnFLxwEVFykWKpraHgsutBMjYJc",10000],["n4QQa6X49seH4oK6Y6ANk66aWsFYGjbF8Y",66384]]}'

In [14]:
send_tx(BTCTEST, btc_acc, "mtnxNt9cnFLxwEVFykWKpraHgsutBMjYJc", 0.0001 )

In [12]:
send_tx(BTCTEST, btc_acc, "mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt", 0.0001 )

In [25]:

w3.eth.getBalance("0x907b31E07E8D9c5504Cc1bf28C27Ec28bF8BE111")


0

In [ ]:
create_tx(ETH,eth_acc,"", 1000)